<a href="https://colab.research.google.com/github/NuryaFahruRosyidin2406/Machine-Learning_Studi-Kasus-Kedua-yaitu-Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pendahuluan Studi Kasus Kedua: Sentiment Analysis

Inilah materi yang akan Anda pelajari pada modul Studi Kasus Kedua: Sentiment Analysis. Di modul kali ini Anda akan mempelajari analisis sentimen media sosial menggunakan data teks dalam Bahasa Indonesia, yaitu dataset IndoNLU. Hingga akhir modul ini, diharapkan Anda dapat:

- Memahami apa itu analisis sentimen
- Memahami contoh penerapan Natural Language Processing
- Memahami data pipeline kasus Natural Language Processing
- Memahami teknik feature engineering untuk representasi teks
- Melakukan analisis sentimen dengan teknik Deep Learning
- Melakukan analisis sentimen dengan algoritma Support Vector Machine

Yuk, lanjut ke materi berikutnya!

# Analisis Sentimen dengan Deep Learning

## Persiapan

Pertama, pada Google Colab Anda, instal PyTorch dengan cara yang telah dijelaskan di materi sebelumnya. Kemudian, karena kita akan menggunakan pre-trained model IndoBert, instal juga transformers. Ia adalah library yang menyediakan general-purpose architectures (BERT, GPT-2, XLM, dll) untuk NLU.

Jalankan kode berikut pada Google Colab Anda satu per satu, masing-masing pada sel yang berbeda.

In [1]:
!pip install torch torchvision
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.5 MB/s eta 0:00:00


Selanjutnya, clone akun github IndoNLU untuk menyimpan dataset pada storage session Google Colab. Eksekusi kode berikut.

In [2]:
!git clone https://github.com/indobenchmark/indonlu

Cloning into 'indonlu'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 500 (delta 115), reused 139 (delta 110), pack-reused 316
Receiving objects: 100% (500/500), 9.45 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (235/235), done.


Perhatikan folder indonlu > dataset > smsa_doc_sentiment_prosa. Pada folder itulah data kita berada. Terlihat beberapa file seperti train_preprocess.tsv (ini adalah data latih yang akan kita gunakan), dan valid_preprocess.tsv (ini adalah data validasi yang akan kita gunakan).

Selanjutnya, impor semua library yang dibutuhkan. Tuliskan kode berikut pada sel selanjutnya.

In [3]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer

from indonlu.utils.forward_fn import forward_sequence_classification
from indonlu.utils.metrics import document_sentiment_metrics_fn
from indonlu.utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

Selanjutnya, definisikan fungsi umum sebagai berikut.

- set_seed : Mengatur dan menetapkan random seed.
- count_param : Menghitung jumlah parameter dalam model
- get_lr : Mengatur learning rate
- metrics_to_string : Mengonversi metriks ke dalam string

Jalankan kode berikut.

In [4]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

Selanjutnya, set random seed dengan menjalankan kode berikut:

In [5]:
# Set random seed
set_seed(19072021)

Anda bebas mengatur random seed asalkan dalam bentuk angka atau integer. Tujuan mengatur random seed adalah agar model memberikan hasil yang sama setiap kali kita melakukan proses training. Untuk memudahkan, set random_seed dengan tanggal saat kita menjalankan kode tersebut.

## Konfigurasi dan Load Pre-trained Model

Tahap selanjutnya adalah proses Load Model dan konfigurasi. Pada tahap ini, kita menggunakan pre-trained model Indobert-base-p1 yang memiliki 124.5 juta parameter. Untuk tipe Indobert lainnya, Anda dapat melihatnya pada tautan berikut.

Model Indobert dibangun berdasarkan general-purpose architecture BERT (Bidirectional Encoder Representation from Transformers).  BERT didesain untuk membantu komputer memahami arti bahasa ambigu dalam teks. Caranya adalah menggunakan teks di sekitarnya untuk membangun konteks.

Mari kita load tokenizer, config, dan inisiasi model dengan kode berikut.

In [6]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setelah kode dijalankan, proses download model akan berlangsung, seperti ini.

Untuk melihat modelnya, kita panggil model yang telah diinisiasi pada kode sebelumnya.

In [7]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

Berikut adalah cuplikan keluaran dari modelnya. Kita bisa lihat model memiliki belasan layer.

Untuk melihat jumlah parameter, jalankan kode berikut.

In [8]:
count_param(model)

124443651

Hasilnya, ada 124.443.651 parameter dalam pre-trained model IndoBert-base-p1.

## Persiapan Dataset Analisis Sentimen

Tahap selanjutnya adalah mempersiapkan dataset. Perhatikan file dan path pada bagian Files di sisi kiri. Temukan tanda titik tiga pada data yang akan Anda gunakan. Kemudian klik tanda titik tiga tersebut dan pilih **Copy path** untuk mendapatkan file path data. Tempatkan file path pada variabel data agar mesin bisa membacanya.

Tulis kode berikut untuk mempersiapkan dataset.

In [9]:
train_dataset_path = '/content/indonlu/dataset/smsa_doc-sentiment-prosa/train_preprocess.tsv'
valid_dataset_path = '/content/indonlu/dataset/smsa_doc-sentiment-prosa/valid_preprocess.tsv'
test_dataset_path = '/content/indonlu/dataset/smsa_doc-sentiment-prosa/test_preprocess_masked_label.tsv'

Setelah menentukan lokasi dataset, kita perlu menyiapkan data dengan Pytorch. PyTorch menyediakan cara terstandarisasi untuk menyiapkan data sebelum melakukan pemodelan. PyTorch menyediakan banyak fitur canggih untuk memproses data.

Di sini, kita akan menggunakan 2 kelas yang disediakan di PyTorch dalam modul torch.utils.data yaitu Dataset dan DataLoader. Disadur dari Pre-Trained Models for NLP Tasks Using PyTorch [39], kelas Dataset adalah sebuah abstract class yang perlu kita extend di PyTorch. Sedangkan, DataLoader adalah inti dari perangkat pemrosesan data di PyTorch. DataLoader menyediakan banyak fungsionalitas untuk mempersiapkan data termasuk berbagai metode sampling, komputasi paralel, dan pemrosesan terdistribusi. Nah, kita akan memindahkan objek dari kelas Dataset ke dalam objek dari kelas DataLoader untuk pemrosesan batch data lebih lanjut.

Untuk menunjukan bagaimana cara mengimplementasikan Dataset dan DataLoader di PyTorch, kita akan melihat lebih dalam pada kelas DocumentSentimentDataset dan DocumentSentimentDataLoader yang disediakan oleh IndoNLU. Anda dapat mengaksesnya di tautan berikut: data_utils.py.

Selanjutnya, kita akan mengimplementasikan kelas DocumentSentimentDataset untuk data loading. Untuk membuat kelas DocumentSentimentDataset yang fungsional, implementasikan 3 fungsi berikut, __init__(self, ...), __getitem__(self, index), dan __len__(self).

Kemudian, implementasikan juga fungsi __getitem__(self, index) dan __len__(self). Sehingga, definisi lengkap dari kelas DocumentSentimentDataset adalah sebagai berikut.

In [10]:
# class DocumentSentimentDataset(Dataset):
#     # Static constant variable
#     LABEL2INDEX = {'positive': 0, 'neutral': 1, 'negative': 2} # Map dari label string ke index
#     INDEX2LABEL = {0: 'positive', 1: 'neutral', 2: 'negative'} # Map dari Index ke label string
#     NUM_LABELS = 3 # Jumlah label

#     def load_dataset(self, path):
#         df = pd.read_csv(path, sep='\t', header=None) # Baca tsv file dengan pandas
#         df.columns = ['text','sentiment'] # Berikan nama pada kolom tabel
#         df['sentiment'] = df['sentiment'].apply(lambda lab: self.LABEL2INDEX[lab]) # Konversi string label ke index
#         return df

#     def __init__(self, dataset_path, tokenizer, *args, **kwargs):
#         self.data = self.load_dataset(dataset_path) # Load tsv file

#         # Assign tokenizer, disini kita menggunakan tokenizer subword dari HuggingFace
#         self.tokenizer = tokenizer

#     def __getitem__(self, index):
#         data = self.data.loc[index,:] # Ambil data pada baris tertentu dari tabel
#         text, sentiment = data['text'], data['sentiment'] # Ambil nilai text dan sentiment
#         subwords = self.tokenizer.encode(text) # Tokenisasi text menjadi subword

#     # Return numpy array dari subwords dan label
#         return np.array(subwords), np.array(sentiment), data['text']

#     def __len__(self):
#         return len(self.data)  # Return panjang dari dataset

In [11]:
# class DocumentSentimentDataLoader(DataLoader):
#     def __init__(self, max_seq_len=512, *args, **kwargs):
#         super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
#         self.max_seq_len = max_seq_len # Assign batas maksimum subword
#         self.collate_fn = self._collate_fn # Assign fungsi collate_fn dengan fungsi yang kita definisikan

#     def _collate_fn(self, batch):
#         batch_size = len(batch) # Ambil batch size
#         max_seq_len = max(map(lambda x: len(x[0]), batch)) # Cari panjang subword maksimal dari batch
#         max_seq_len = min(self.max_seq_len, max_seq_len) # Bandingkan dengan batas yang kita tentukan sebelumnya

#     # Buat buffer untuk subword, mask, dan sentiment labels, inisialisasikan semuanya dengan 0
#         subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
#         mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
#         sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)

#     # Isi semua buffer
#         for i, (subwords, sentiment, raw_seq) in enumerate(batch):
#             subwords = subwords[:max_seq_len]
#             subword_batch[i,:len(subwords)] = subwords
#             mask_batch[i,:len(subwords)] = 1
#             sentiment_batch[i,0] = sentiment

#     # Return subword, mask, dan sentiment data
#         return subword_batch, mask_batch, sentiment_batch

Mari kita lanjutkan dengan mendefinisikan variabel untuk kedua kelas tadi. Eksekusi kode berikut.

In [12]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=True)
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Setelah proses penetapan variabel selesai, mari kita print hasilnya pada salah satu sampel data, misalnya train_dataset[0]. Terapkan kode berikut.

In [13]:
print(train_dataset[0])

(array([    2,  6540,    92,  2970,   213,  4259,  3553,   899,    34,
         259,  5590,   262,  2558,   386,   899,  1687,    26,  1574,
       30470,   899,  3310, 30468, 22130, 30360,  6123,  6368, 30468,
       22130, 30360,  2652,  1746, 30468,  8869,  6540,    34,  6315,
        1622,  1256,  8949,   899, 30468,  4222,  1622,   752,   245,
         295,  2083, 30470,  2346,  7107,   300, 30470,   405,   724,
        5189, 30470,   843, 17464,   899,   540, 10989,  3331,  1107,
       30468,   119,  3221,    79,    34,  2170,    98,  9167, 30457,
           3]), array(0), 'warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !')


Langkah berikutnya adalah mendefinisikan variabel, misalnya w2i dan i2w untuk menempatkan DocumentSentimentDataset.LABEL2INDEX dan DocumentSentimentDataset.INDEX2LABEL.

In [14]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'positive': 0, 'neutral': 1, 'negative': 2}
{0: 'positive', 1: 'neutral', 2: 'negative'}


# Uji Model dengan Contoh Kalimat

Berikutnya, kita akan mengecek model pada contoh kalimat. Tuliskan kode berikut.

In [15]:
text = 'Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita | Label : positive (52.480%)


Saat dijalankan, kode memberikan keluaran seperti ini.

Padahal, kita dapat mengidentifikasi kalau teks tersebut seharusnya masuk dalam kategori sentimen positif. Oleh karena itu, mari kita lakukan proses Fine Tuning dan Evaluasi.

# Fine Tuning dan Evaluasi
Pertama, definisikan optimizer. Di sini kita menggunakan Adam optimizer pada library PyTorch. Selanjutnya, jika Anda menjalankan proyek di Google Colab, pastikan pengaturan runtime telah diubah ke GPU. Anda dapat mengubah pengaturannya pada menu toolbar. Klik Runtime > Change Runtime Type > Hardware Accelerator pilih GPU, lalu klik Save.

Sekarang, jalankan kode berikut. Panggil juga fungsi cuda() agar proses pelatihan berjalan cepat dengan bantuan gpu.

In [16]:
optimizer = optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

Selanjutnya, kita akan melatih model dengan jumlah epoch = 5. Tahapan yang akan kita lakukan, antara lain:

- Proses pelatihan dan pembaruan model
- Kalkulasi metriks pelatihan
- Evaluasi pada data validasi
- Kalkulasi matriks validasi

Jalankan kode berikut.

In [17]:
# Train
n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))

    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
(Epoch 1) TRAIN LOSS:0.3364 LR:0.00000300: 100%|██████████| 344/344 [02:34<00:00,  2.22it/s]


(Epoch 1) TRAIN LOSS:0.3364 ACC:0.87 F1:0.83 REC:0.80 PRE:0.87 LR:0.00000300


  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
VALID LOSS:0.1967 ACC:0.92 F1:0.89 REC:0.89 PRE:0.90: 100%|██████████| 40/40 [00:08<00:00,  4.97it/s]


(Epoch 1) VALID LOSS:0.1967 ACC:0.92 F1:0.89 REC:0.89 PRE:0.90


  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
(Epoch 2) TRAIN LOSS:0.1583 LR:0.00000300: 100%|██████████| 344/344 [02:37<00:00,  2.18it/s]


...

(Epoch 4) VALID LOSS:0.1777 ACC:0.94 F1:0.92 REC:0.91 PRE:0.93


  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
(Epoch 5) TRAIN LOSS:0.0661 LR:0.00000300: 100%|██████████| 344/344 [02:39<00:00,  2.15it/s]


(Epoch 5) TRAIN LOSS:0.0661 ACC:0.98 F1:0.98 REC:0.97 PRE:0.98 LR:0.00000300


  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
VALID LOSS:0.1852 ACC:0.94 F1:0.91 REC:0.90 PRE:0.92: 100%|██████████| 40/40 [00:08<00:00,  4.92it/s]

(Epoch 5) VALID LOSS:0.1852 ACC:0.94 F1:0.91 REC:0.90 PRE:0.92


Jika berhasil, model akan melakukan proses training dengan 5 epochs.

Selanjutnya, kita akan melakukan evaluasi pada data test.

In [18]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

print(df)
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

print(df)

  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 16/16 [00:03<00:00,  4.77it/s]


     index     label
0        0  negative
1        1  negative
2        2  negative
3        3  negative
4        4  negative
..     ...       ...
495    495   neutral
496    496   neutral
497    497  positive
498    498  positive
499    499  positive

[500 rows x 2 columns]


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 16/16 [00:02<00:00,  5.68it/s]

     index     label
0        0  negative
1        1  negative
2        2  negative
3        3  negative
4        4  negative
..     ...       ...
495    495   neutral
496    496   neutral
497    497  positive
498    498  positive
499    499  positive

[500 rows x 2 columns]


Hasil evaluasi akan memberikan dataframe sebagai berikut:

# Prediksi Sentimen

Sekarang, saatnya kita menguji model pada beberapa contoh kalimat. Jalankan kode berikut dan ubah variabel **text** dengan beberapa contoh kalimat yang Anda sukai.

In [19]:
text = 'Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita | Label : positive (99.394%)


Kali ini, teks "Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita" berhasil mendapatkan label positif dengan 99.5% akurasi.

Ubah kalimat pada variabel text dengan beberapa contoh kalimat di bawah ini.

1. “Ronaldo pergi ke Mall Grand Indonesia membeli cilok”
2. “Sayang, aku marah”
3. “Merasa kagum dengan toko ini tapi berubah menjadi kecewa setelah transaksi”

Kalimat nomor dua sengaja dibuat seperti itu dengan maksud untuk mengecoh mesin. Kata “sayang” biasanya berkonotasi positif. Tapi dalam kalimat tersebut diikuti oleh kata “marah”.

Demikian juga dengan kalimat nomor 3, “kagum” dan “kecewa” adalah dua kata yang masing-masing berkonotasi positif dan negatif. Jika dilihat dari konteks kalimatnya, jelas kedua kalimat ini bersentimen negatif. Manusia bisa langsung mengenalinya sebagai sentimen negatif. Namun, bagaimana dengan mesin yang telah dilatih dengan model kita?

In [20]:
# Text 1
text = 'Ronaldo pergi ke Mall Grand Indonesia membeli cilok'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Ronaldo pergi ke Mall Grand Indonesia membeli cilok | Label : neutral (99.559%)


In [21]:
# Text 2
text = 'Sayang, aku marah'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Sayang, aku marah | Label : negative (99.753%)


In [22]:
# Text 3
text = 'Merasa kagum dengan toko ini tapi berubah menjadi kecewa setelah transaksi'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Merasa kagum dengan toko ini tapi berubah menjadi kecewa setelah transaksi | Label : negative (99.634%)
